Install happy transformer

In [1]:
!pip install happytransformer
from IPython.display import clear_output
clear_output()

In [2]:
!pip install transformers[torch]

In [3]:
!pip install accelerate -U

Import libraries

In [4]:
import csv
from datasets import load_dataset
from happytransformer import TTSettings
from happytransformer import TTTrainArgs
from happytransformer import HappyTextToText

In [5]:
happy_tt = HappyTextToText("T5", "t5-base")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Load dataset

In [6]:
train_data = load_dataset("jfleg", split='validation[:]')
test_data = load_dataset("jfleg", split='test[:]')

In [7]:
for data in train_data["corrections"][:2]:
    print(data)
    print(data[0])
    print("==================================================")

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 


Data preprocessing:
Data yang didapet harus dibuat formatnya agar sesuai dengan model happy transformer (csv), Input=grammar salah, Output=grammar benernya

In [8]:
def create_csv(path, dataset):
    with open(path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar:" + case["sentence"]
            for correction in case["corrections"]:
                if input_text and correction:
                    writer.writerow([input_text, correction])


In [9]:
create_csv("train_data.csv", train_data)
create_csv("test_data.csv", test_data)

Training model

In [10]:
args = TTTrainArgs(batch_size = 8)
happy_tt.train("train_data.csv", args=args)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3016 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.591300
1000,0.441800


In [12]:
beam_settings = TTSettings(num_beams = 5, min_length=1, max_length=20)

In [19]:
happy_tt.save("model")

In [35]:
eval = happy_tt.eval("test_data.csv")
print("Loss: ", eval.loss)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Loss:  0.4484277367591858


In [57]:
def generate(input):
    input = "grammar: " + input
    beam_settings = TTSettings(num_beams=5, min_length=1, max_length=20)
    predicted = happy_tt.generate_text(input, args=beam_settings)
    #correct = predicted.text.strip()
    return predicted.text

In [52]:
user_input = input("Kalimat anda: ")
user_input = "grammar: " + user_input

correction = generate(user_input)
print("Kalimat dengan grammar yang benar: ", correction)

Kalimat anda: i like for walks
Kalimat dengan grammar yang benar:  I like to go for walks.


In [51]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.5 MB/s eta 0:00:00
    

In [58]:
import gradio as gr

app_input = gr.inputs.Textbox(lines=2, placeholder='Kalimat anda...')
interface = gr.Interface(fn=generate, inputs=app_input, outputs='text', title='Project KRS Semester 2')
interface.launch()

<ipython-input-58-564681e6695d>:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  app_input = gr.inputs.Textbox(lines=2, placeholder='Kalimat anda...')
<ipython-input-58-564681e6695d>:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  app_input = gr.inputs.Textbox(lines=2, placeholder='Kalimat anda...')
<ipython-input-58-564681e6695d>:3: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  app_input = gr.inputs.Textbox(lines=2, placeholder='Kalimat anda...')


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [60]:
!tar -czvf koda2.tar.gz /content/model

tar: Removing leading `/' from member names
/content/model/
/content/model/generation_config.json
/content/model/tokenizer.json
/content/model/pytorch_model.bin
/content/model/special_tokens_map.json
/content/model/config.json
/content/model/tokenizer_config.json
/content/model/spiece.model


In [62]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
!cp /content/koda2.tar.gz "/content/drive/MyDrive/what/koda2.tar.gz"